## 必要なライブラリーをインストール
### ※ 実行環境にインストールされていない場合のみ、以下のコメント(#)を外して実行してください

In [9]:
# %pip install lapx # オブジェクトトラッキング（追跡）に必要な線形割り当てライブラリ

In [11]:
# %pip install labelImg # アノテーション（教師データ作成）用ツール

In [13]:
# %pip install pandas # データ集計・CSV操作用

In [15]:
# %pip install ultralytics lapx # YOLOv8をしようするための公式ライブラリ

## データセットの構築と前処理
### 学習用・検証用データの分割

In [44]:
import os
import shutil
import glob
import random

# 1. パスの設定
base_dir = os.getcwd()
dataset_dir = os.path.join(base_dir, "dataset")
train_img_dir = os.path.join(dataset_dir, "train", "images")
train_lbl_dir = os.path.join(dataset_dir, "train", "labels")
val_img_dir = os.path.join(dataset_dir, "val", "images")
val_lbl_dir = os.path.join(dataset_dir, "val", "labels")

# 2. フォルダ作成
for d in [train_img_dir, train_lbl_dir, val_img_dir, val_lbl_dir]:
    os.makedirs(d, exist_ok=True)

print("--- データの整理を開始します ---")

# 3. 【リセット】val の中身を全て train に戻す
val_images = glob.glob(os.path.join(val_img_dir, "*.jpg"))
if len(val_images) > 0:
    print(f"現在 val にある {len(val_images)}枚 を一度 train に戻してリセットします...")
    for img_path in val_images:
        basename = os.path.basename(img_path)
        txt_name = basename.replace(".jpg", ".txt")
        
        # 移動元
        src_img = os.path.join(val_img_dir, basename)
        src_txt = os.path.join(val_lbl_dir, txt_name)
        
        # 移動先
        dst_img = os.path.join(train_img_dir, basename)
        dst_txt = os.path.join(train_lbl_dir, txt_name)
        
        shutil.move(src_img, dst_img)
        if os.path.exists(src_txt):
            shutil.move(src_txt, dst_txt)

# capture_*.txt のクラスID修正
# ホームベースのデータなのにクラスIDが '0' (ボール) になっている場合、'1' に修正する
print("ラベルファイルのチェックと修正を行います...")
capture_txts = glob.glob(os.path.join(train_lbl_dir, "capture_*.txt"))
fixed_count = 0

for txt_path in capture_txts:
    with open(txt_path, 'r') as f:
        lines = f.readlines()
    
    new_lines = []
    modified = False
    for line in lines:
        parts = line.strip().split()
        # 先頭が '0' (ボール) だったら '1' (ホームベース) に書き換える
        if len(parts) > 0 and parts[0] == '0':
            parts[0] = '1'
            new_line = " ".join(parts) + "\n"
            new_lines.append(new_line)
            modified = True
        else:
            new_lines.append(line)
            
    if modified:
        with open(txt_path, 'w') as f:
            f.writelines(new_lines)
        fixed_count += 1

if fixed_count > 0:
    print(f"修正完了: {fixed_count}個のファイルを '1: home_plate' に修正しました。")
else:
    print("修正不要: すべてのファイルが正しいクラスIDでした。")

# 4. ファイル名でグループ分け
all_images = glob.glob(os.path.join(train_img_dir, "*.jpg"))

ball_images = []       # 数字だけのファイル
plate_images = []      # capture_ から始まるファイル

for img_path in all_images:
    filename = os.path.basename(img_path)
    if filename.startswith("capture_"):
        plate_images.append(img_path)
    else:
        ball_images.append(img_path)

print(f"ボール画像: {len(ball_images)}枚")
print(f"ホームベース画像: {len(plate_images)}枚")

# 5. それぞれから 20% を val に移動（ランダム抽出）
# 偏りを防ぐためにランダムに

def move_to_val(image_list, ratio=0.2):
    # ランダムにシャッフル
    random.shuffle(image_list)
    
    # 移動する枚数を計算
    num_move = int(len(image_list) * ratio)
    if len(image_list) > 0 and num_move == 0:
        num_move = 1
        
    count = 0
    for img_path in image_list[:num_move]:
        basename = os.path.basename(img_path)
        txt_name = basename.replace(".jpg", ".txt")
        
        src_img = os.path.join(train_img_dir, basename)
        src_txt = os.path.join(train_lbl_dir, txt_name)
        
        dst_img = os.path.join(val_img_dir, basename)
        dst_txt = os.path.join(val_lbl_dir, txt_name)
        
        if os.path.exists(src_img) and os.path.exists(src_txt):
            shutil.move(src_img, dst_img)
            shutil.move(src_txt, dst_txt)
            count += 1
    return count

moved_balls = move_to_val(ball_images)
moved_plates = move_to_val(plate_images)

print("-" * 30)
print(f"移動完了:")
print(f"  ボール: {moved_balls}枚 を val へ")
print(f"  ホームベース: {moved_plates}枚 を val へ")

# 6. data.yaml の更新
yaml_path = os.path.join(dataset_dir, "data.yaml")
yaml_content = f"""
path: {dataset_dir}
train: train/images
val: val/images

# クラス名を定義
names:
  0: ball
  1: home_plate
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"設定ファイルを更新しました: {yaml_path}")

--- データの整理を開始します ---
現在 val にある 66枚 を一度 train に戻してリセットします...
ラベルファイルのチェックと修正を行います...
修正完了: 30個のファイルを '1: home_plate' に修正しました。
ボール画像: 300枚
ホームベース画像: 30枚
------------------------------
移動完了:
  ボール: 60枚 を val へ
  ホームベース: 6枚 を val へ
設定ファイルを更新しました: /Users/chise1/Desktop/Baseball_AI/dataset/data.yaml


## モデルの学習
### 物体検出AI（YOLOv8）の学習

In [46]:
import glob
import os

# runs/detect 以下の train フォルダをすべて取得
list_of_files = glob.glob('runs/detect/train*') 
# 作成日時が新しい順に並び替え
latest_folder = max(list_of_files, key=os.path.getctime)
# best.pt のパスを作成
best_model_path = os.path.join(latest_folder, 'weights', 'best.pt')

from ultralytics import YOLO
import os

# ==============================================================================
# モデルの学習 (Model Training)
# 事前学習済みモデル(yolov8n.pt)をベースに、転移学習を行います。
# ==============================================================================

# 1. モデルの準備
model = YOLO('yolov8n.pt')

# 2. 設定ファイルのパス
yaml_path = os.path.abspath("dataset/data.yaml")

print(f"学習を開始します... データ: {yaml_path}")

# 3. 学習実行
# epochs=100: 精度を確保するため、十分な回数の学習を行います。
# (開発時は epochs=10 程度で動作確認を行い、パイプラインに問題ないことを確認済みです)
results = model.train(
    data=yaml_path, 
    epochs=100, 
    imgsz=640
)

print("全学習完了！")

学習を開始します... データ: /Users/chise1/Desktop/Baseball_AI/dataset/data.yaml
New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.233 🚀 Python-3.12.7 torch-2.9.1 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/chise1/Desktop/Baseball_AI/dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, m

In [50]:
import glob
import os

# =====================================================================
# 最新モデルの自動取得
# 学習を繰り返すと train, train2, train3... とフォルダが増えていくため、
# 手動でパスを書き換えなくて済むように、作成日時が最新のものを自動的に選択するロジック。
# =====================================================================

# 1. 'runs/detect/' 以下にある 'train' で始まるフォルダを全て取得
list_of_files = glob.glob('runs/detect/train*')

# 2. 取得したフォルダの中から、作成日時(getctime)が最も新しいものを特定
#    latest_folder には 'runs/detect/train7' のようなパスが入る
latest_folder = max(list_of_files, key=os.path.getctime)

# 3. パスを結合して、学習済みモデル(best.pt)の絶対パスを生成
#    例: 'runs/detect/train7/weights/best.pt'
best_model_path = os.path.join(latest_folder, 'weights', 'best.pt')

print(f"★今回使うモデル: {best_model_path}")

★今回使うモデル: runs/detect/train9/weights/best.pt


# 奥行きを決めるための処理

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import os

# ================= 設定エリア ===========================================
MODEL_PATH = 'runs/detect/train9/weights/best.pt'
VIDEO_PATH = 'video/3.mp4'
# =====================================================================

model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

# 1. 回転が必要かどうかの判定（全データ解析と同じロジック）
w_org = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h_org = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

rotate_needed = False
if w_org > h_org:
    rotate_needed = True
    print(f"動画サイズ: {w_org}x{h_org} -> 回転あり")
else:
    print(f"動画サイズ: {w_org}x{h_org} -> 回転なし")

print("--- コマ送り計測モード開始 ---")
print("「n」キー: 次のフレームへ")
print("「q」キー: 終了")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("動画終了")
        break

    # 2. 全データ解析と同じ回転処理を適用
    if rotate_needed:
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

    # 3. 推論
    results = model.track(frame, persist=True, conf=0.1, verbose=False)
    
    ball_w = 0
    plate_w = 0
    
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xywh.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        
        # ボールとホームベースを分けて探す
        ball_candidates = []
        plate_candidates = []

        for box, cls_id in zip(boxes, classes):
            # Class 0 = Ball, Class 1 = Home Plate
            if int(cls_id) == 0:
                ball_candidates.append(box)
            elif int(cls_id) == 1:
                plate_candidates.append(box)

        # ★修正点：ボールの中で一番大きいものを探す（ホームベースと混同しない）
        if len(ball_candidates) > 0:
            # 面積(w*h)が一番大きいボールを選ぶ
            best_ball = max(ball_candidates, key=lambda x: x[2] * x[3])
            bx, by, bw, bh = best_ball
            ball_w = bw
            
            # 描画：ボール（黄色）
            cv2.circle(frame, (int(bx), int(by)), 5, (0, 255, 255), -1)
            # サイズを表示
            text = f"Ball W: {bw:.1f}"
            cv2.rectangle(frame, (int(bx)+10, int(by)-20), (int(bx)+160, int(by)+5), (0,0,0), -1)
            cv2.putText(frame, text, (int(bx)+10, int(by)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        # 参考：ホームベースも表示（緑色）
        if len(plate_candidates) > 0:
            best_plate = max(plate_candidates, key=lambda x: x[2] * x[3])
            px, py, pw, ph = best_plate
            plate_w = pw
            
            cv2.rectangle(frame, (int(px-pw/2), int(py-ph/2)), (int(px+pw/2), int(py+ph/2)), (0, 255, 0), 2)
            cv2.putText(frame, f"Plate W: {pw:.1f}", (int(px)-40, int(py)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # 情報表示
    cv2.putText(frame, "Press 'n' -> Next", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # 画面に情報を出す（コンソールにも出す）
    if ball_w > 0:
        print(f"検出中: Ball幅={ball_w:.2f} / Plate幅={plate_w:.2f}")

    cv2.imshow('Check Size', frame)

    while True:
        key = cv2.waitKey(0) & 0xFF
        if key == ord('n'):
            break
        elif key == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            # 強制終了
            os._exit(0)

cap.release()
cv2.destroyAllWindows()

動画サイズ: 720x1280 -> 回転なし
--- コマ送り計測モード開始 ---
「n」キー: 次のフレームへ
「q」キー: 終了


# ボールの判定位置を決めたコード

### 1データの解析

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

# ==============================================================================
# 設定エリア
# 開発したストライク判定AIの動作パラメータを定義します。
# これらの値は、実際の野球のルールや物理的なサイズ比率に基づいて調整しています。
# ==============================================================================

# 解析対象の動画ファイルと、学習済みAIモデルのパス
VIDEO_PATH = 'video/1.mp4' 
MODEL_PATH = 'runs/detect/train9/weights/best.pt'

# --- ストライクゾーンの形状設定 ---
# ホームベースの幅を基準(1.0)として、ゾーンの大きさを決定
# 1. 横幅: ベース幅と同じにするため 1.0
ZONE_W_RATIO = 1.0

# 2. 高さ: 高校球児の平均身長(約171cm)の胸〜膝の長さを考慮して設定 (1.09倍)
ZONE_H_RATIO = 1.09

# 3. 高さ位置(オフセット): 地面から膝下までの高さを調整 (1.03倍)
ZONE_OFFSET_RATIO = 1.03

# --- 判定ロジックの設定 ---
# 4. 奥行きの判定基準
# カメラから見て、ボールの幅がホームベース幅の「0.18倍」になった瞬間を
# 「ホームベース上を通過した」とみなして判定
BALL_SIZE_RATIO = 0.18

# 5. 判定の厳しさ (True: 中心がゾーン内 / False: 端がかかればOK)
# 野球のルールでは「少しかすってもストライク」なので False に設定
STRICT_MODE = False

# 誤検知対策: 画面上部20%（天井など）はAIの検出対象外とする
IGNORE_TOP_RATIO = 0.2

# ==============================================================================
# ストライク・ボール自動判定クラス
# ==============================================================================
class StrikeBallJudgeFinal:
    def __init__(self, model_path, video_path):
        """
        初期化処理: AIモデルの読み込みと変数のセットアップ
        """
        print(f"モデル読み込み中...: {model_path}")
        if not os.path.exists(model_path):
            raise FileNotFoundError("学習済みモデルが見つかりません")
            
        # YOLOv8モデルのロード
        self.model = YOLO(model_path)
        self.video_path = video_path
        
        # 判定結果を保存する変数 (STRIKE / BALL)
        self.final_judgment = None
        self.result_color = (0, 0, 0) # 結果表示用の色 (緑/赤)
        
        # ゾーン計算を安定させるためのリスト（移動平均用）
        self.plate_width_avg = []
        self.fixed_plate_width = 0
        self.zone_coords = None
        
        # ボールの追跡用
        self.prev_ball = None       # 1フレーム前のボール情報
        self.ball_trajectory = []   # 軌跡を描画するための座標リスト
        self.judged_ball_info = None # 判定された瞬間のボール情報

    def process_video(self):
        """
        動画を1フレームずつ読み込み、解析・判定・描画を行うメインループ
        """
        cap = cv2.VideoCapture(self.video_path)
        
        # 動画のサイズを取得
        w_org = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h_org = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # スマホの縦撮り動画などの場合、90度回転させる必要があるか判定
        # (横幅 > 高さ なら横向きとみなして回転フラグを立てる)
        if w_org > h_org:
            rotate = True
            screen_w, screen_h = h_org, w_org # 回転後は幅と高さが入れ替わる
        else:
            rotate = False
            screen_w, screen_h = w_org, h_org

        # 天井照明などを誤検出しないための無視ライン (上から20%)
        ignore_y = int(screen_h * IGNORE_TOP_RATIO)
        
        print("--- 解析開始 ---")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break # 動画終了
            
            # 必要なら画像を90度回転
            if rotate: frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

            # --- 1. AIによる物体検出 ---
            # trackメソッドで物体を検出し、IDを割り当てて追跡する
            results = self.model.track(frame, persist=True, conf=0.15, verbose=False)
            
            curr_ball = None
            curr_plate = None

            if results[0].boxes.id is not None:
                # 検出されたバウンディングボックス(座標とサイズ)を取得
                boxes = results[0].boxes.xywh.cpu().numpy()
                classes = results[0].boxes.cls.cpu().numpy()
                
                for box, cls_id in zip(boxes, classes):
                    bx, by, bw, bh = box
                    
                    # 画面上部(天井エリア)なら無視
                    if by < ignore_y: continue 

                    # クラスID 0: ボール
                    if int(cls_id) == 0:
                        # 複数ある場合は、一番大きく映っている(=手前にある)ボールを採用
                        if curr_ball is None or bw > curr_ball[2]:
                            curr_ball = box
                            
                    # クラスID 1: ホームベース
                    elif int(cls_id) == 1:
                        if curr_plate is None or bw > curr_plate[2]:
                            curr_plate = box

            # --- 2. ストライクゾーンの動的生成 ---
            # ホームベースが検出されていれば、その幅を「定規」としてゾーンを計算する
            if curr_plate is not None:
                px, py, pw, ph = curr_plate
                
                # 検出ブレを防ぐため、過去数フレームの平均をとってサイズを安定させる
                self.plate_width_avg.append(pw)
                if len(self.plate_width_avg) > 10: self.plate_width_avg.pop(0)
                self.fixed_plate_width = sum(self.plate_width_avg) / len(self.plate_width_avg)

                # ベース幅(base_w)を基準に各座標を計算
                base_w = self.fixed_plate_width
                zw = int(base_w * ZONE_W_RATIO)
                zh = int(zw * ZONE_H_RATIO)
                
                # 下限(zb)はベース位置(py)からオフセット分上にずらす
                zb = int(py - (base_w * ZONE_OFFSET_RATIO))
                # 上限(zt)は下限から高さを引く
                zt = zb - zh
                zx = int(px)
                
                # ゾーンの左端(zl)と右端(zr)
                zl, zr = zx - zw//2, zx + zw//2
                self.zone_coords = (zl, zt, zr, zb)

            # ゾーンの描画 (緑色の枠線)
            if self.zone_coords:
                zl, zt, zr, zb = self.zone_coords
                cv2.rectangle(frame, (zl, zt), (zr, zb), (0, 255, 0), 2)
                cv2.putText(frame, "Zone", (zl, zt - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            # --- 3. ボールの軌跡表示 ---
            if curr_ball is not None:
                bx, by, bw, bh = curr_ball
                # 軌跡リストに現在位置を追加
                self.ball_trajectory.append((int(bx), int(by)))
                # 現在のボール位置に円を描画
                cv2.circle(frame, (int(bx), int(by)), 5, (0, 255, 255), -1)

            # 過去の軌跡を小さな点で描画
            for pt in self.ball_trajectory:
                cv2.circle(frame, pt, 3, (0, 0, 255), -1)

            # --- 4. ストライク/ボール判定ロジック ---
            # まだ判定が出ておらず、ゾーンが確定している場合に実行
            if self.final_judgment is None and self.zone_coords and self.fixed_plate_width > 0:
                
                # 通過とみなす基準サイズ (ベース幅 x 0.18)
                target_w = self.fixed_plate_width * BALL_SIZE_RATIO
                
                if curr_ball is not None:
                    bx, by, bw, bh = curr_ball
                    
                    # ボールの大きさが基準を超えたら「ホームベース到達」とみなす
                    if bw >= target_w:
                        ball_radius = bw / 2
                        
                        # 【補間処理】
                        # フレーム間でボールが急に大きくなった場合、
                        # ちょうど基準サイズになった瞬間の座標を計算(線形補間)して精度を高める
                        if self.prev_ball is not None:
                            prev_x, prev_y, prev_w, prev_h = self.prev_ball
                            if prev_w < target_w:
                                # 比率計算
                                ratio = (target_w - prev_w) / (bw - prev_w)
                                # 補間座標の算出
                                cross_y = prev_y + (by - prev_y) * ratio
                                cross_x = prev_x + (bx - prev_x) * ratio
                                # 判定実行
                                self._judge_point(cross_x, cross_y, ball_radius)
                            else:
                                self._judge_point(bx, by, ball_radius)
                        else:
                            self._judge_point(bx, by, ball_radius)
                    
                    # 前回のボール情報を更新
                    self.prev_ball = curr_ball

            # --- 5. 結果の表示 ---
            # 判定された瞬間のボールをピンク色で表示
            if self.judged_ball_info:
                jx, jy, jr = self.judged_ball_info
                cv2.circle(frame, (int(jx), int(jy)), int(jr), (255, 0, 255), 2)
                cv2.circle(frame, (int(jx), int(jy)), 4, (0, 0, 255), -1)

            # 画面中央に大きく結果を表示
            if self.final_judgment:
                # 枠線
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), self.result_color, 10)
                # テキスト
                cv2.putText(frame, self.final_judgment, (50, 200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2.5, self.result_color, 5)

            # 画像を表示
            cv2.imshow('Final Judge', frame)
            
            # 'q'キーが押されたら中断
            if cv2.waitKey(1) & 0xFF == ord('q'): break

        # 動画終了後の待機処理 (ウィンドウを閉じない)
        while True:
            if cv2.waitKey(100) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break
            if cv2.getWindowProperty('Final Judge', cv2.WND_PROP_VISIBLE) < 1:
                cap.release()
                break
        cv2.waitKey(1)

    def _judge_point(self, x, y, radius):
        """
        座標(x,y)がストライクゾーンに入っているか判定し、結果を確定させる関数
        """
        zl, zt, zr, zb = self.zone_coords
        
        # STRICT_MODEによって判定基準を変える
        if STRICT_MODE:
            # 厳密モード: ボールの中心がゾーン内にあるか
            is_y_in = (zt <= y <= zb)
            is_x_in = (zl <= x <= zr)
        else:
            # 通常モード: ボールの端がゾーンにかかっていればOK（野球のルール準拠）
            is_y_in = (y + radius >= zt) and (y - radius <= zb)
            is_x_in = (x + radius >= zl) and (x - radius <= zr)

        # 判定結果の保存
        if is_y_in and is_x_in:
            self.final_judgment = "STRIKE!"
            self.result_color = (0, 255, 0) # 緑色
        else:
            self.final_judgment = "BALL"
            self.result_color = (0, 0, 255) # 赤色
        
        # 判定位置を保存
        self.judged_ball_info = (x, y, radius)
        print(f"★判定確定: {self.final_judgment}")

# プログラムの実行
# 指定したモデルと動画を使って判定を開始する
if __name__ == "__main__":
    judge = StrikeBallJudgeFinal(model_path=MODEL_PATH, video_path=VIDEO_PATH)
    judge.process_video()

モデル読み込み中...: runs/detect/train9/weights/best.pt
--- 解析開始 ---
★判定確定: STRIKE!
